# ETL dos registros do Banco de Dados 

## 1. Importações

#### Para este processo de importação vamos precisar importar 2 bibliotecas do Python. A biblioteca `Psycopg2` vai fazer a conexão com o banco de dados e a biblioteca `Pandas` vai nos permitir trabalhar os dados de maneira tabular, onde vai facilitar todo o processo de desde a extração, limpeza, modificação e o processo de salvar os dados em um arquivo de Excel.

In [1]:
import psycopg2
import pandas as pd
import re

## 2. Acessando os dados

#### Como havia instalado o Postgres em meu computador, para acessar o banco de dados foi necessário somente instalar a extensão `PostegreSQL Explorer` no VS Code. Depois foi necessário conectar meu banco de dados com a extensão informando os dados de host, database, user e passaword.

#### Logo após a conexão utilizei a biblioteca `Psycopg2`para ter acesso aos dados. Utilizei a função `execute()` para fazer uma consulta em cada uma das tabelas do banco de dados, e utilizei a função `fetchall()` para armazenar todos os registros em uma variável.

#### Por fim fechei a conexão com o banco de dados.

In [2]:
connection = psycopg2.connect(host="localhost", database="Ultracar", user="postgres", password="admin")

cursor = connection.cursor()

cursor.execute("SELECT * FROM clientes")
clientes = cursor.fetchall()

cursor.execute("SELECT * FROM fornecedores")
fornecedores = cursor.fetchall()

cursor.execute("SELECT * FROM funcionarios")
funcionarios = cursor.fetchall()

cursor.execute("SELECT * FROM orcamentos")
orcamentos = cursor.fetchall()

cursor.execute("SELECT * FROM ordens_servico")
ordens_servico = cursor.fetchall()

cursor.execute("SELECT * FROM produtos")
produtos = cursor.fetchall()

cursor.execute("SELECT * FROM produtos_orcamento")
produtos_orcamento = cursor.fetchall()

cursor.execute("SELECT * FROM servicos")
servicos = cursor.fetchall()

cursor.execute("SELECT * FROM veiculos")
veiculos = cursor.fetchall()


cursor.close()
connection.close()

## 3. Criando os Dataframes

#### Uitilizando a biblioteca `Pandas`, criei todos os dataframes adicionando os títulos de cada coluna. Utilizei as funções `head()` e `info()` para visualizar os dados e identificar algum problema, retirei as funções ao final desta etapa para não poluir visualmente o notebook.  

In [3]:
clientes = pd.DataFrame(clientes, columns=["id_cliente", "nome", "telefone", "email", "endereco", "cidade", "estado", "cep", "data_nascimento", "genero", "status_cliente"])

In [4]:
fornecedores = pd.DataFrame(fornecedores, columns=["id_fornecedor", "nome_fornecedor", "cnpj", "telefone", "email", "endereco", "contato_responsavel"])
funcionarios = pd.DataFrame(funcionarios, columns=["id_funcionario", "nome_funcionario", "cargo", "salario", "data_contratacao", "data_nascimento", "genero", "telefone", "email"])
orcamentos = pd.DataFrame(orcamentos, columns=["id_orcamento", "data_orcamento", "valor_total", "status_orcamento", "id_cliente", "id_veiculo"])
ordens_servico = pd.DataFrame(ordens_servico, columns=["id_ordem_servico", "data_abertura", "data_fechamento", "status_os", "id_funcionario", "id_veiculo", "id_orcamento", "valor_total"])
produtos = pd.DataFrame(produtos, columns=["id_produto", "nome_produto", "descricao", "preco_custo", "preco_venda", "id_fornecedor"])
produtos_orcamento = pd.DataFrame(produtos_orcamento, columns=["id_produto_orcamento", "tipo_item", "id_item", "quantidade", "preco_unitario", "id_orcamento"])
servicos = pd.DataFrame(servicos, columns=["id_servico", "descricao_servico", "preco_servico", "tempo_estimado"])
veiculos = pd.DataFrame(veiculos, columns=["id_veiculo", "placa", "modelo", "marca", "ano_fabricacao", "cor", "tipo_combustivel", "id_cliente"])


## 4. Funções para normalizar os dados

#### Para facilitar o processo de normalização dos dados, resolvi criar funções para evitar reutilização de contigo.


In [5]:
# Função que deixa a primeira letra de cada palavra em maiúscula

def normalizar_nome(nome):
    clientes["nome"] = clientes["nome"].str.title()
    return nome


In [6]:
# Função que normaliza o telefone de acordo com a cidade.

# 1. Substitui espaços em branco
# 2. Remove caracteres caracteres especiais
# 3. Remove o primeiro dígito se o telefone tiver 12 dígitos ou começar com 0
# 4. Adiciona o DDD de acordo com a cidade
# 5. Adiciona o hífen e os parênteses.

def normalizar_telefone_com_cidade(df, telefone_col, cidade_col):
    df[telefone_col] = df[telefone_col].str.replace(' ', '')
    df[telefone_col] = df[telefone_col].apply(lambda x: re.sub(r'\D', '', x))
    df[telefone_col] = df[telefone_col].apply(lambda x: x[1:] if len(x) == 12 or x.startswith('0') else x)
    def adicionar_ddd(telefone, cidade):
        if len(telefone) == 9:
            cidades = {
                "Belo Horizonte": "31",
                "São Paulo": "11",
                "Rio de Janeiro": "21",
                "Vitória": "27",
                "Curitiba": "41",
                "Porto Alegre": "51",
                "Florianópolis": "48",
                "Brasília": "61",
                "Goiânia": "62",
                "Cuiabá": "65",
                "Campo Grande": "67",
                "Rio Branco": "68",
                "Porto Velho": "69",
                "Manaus": "92",
                "Belém": "91",
                "Macapá": "96",
                "Palmas": "63",
                "Boa Vista": "95",
                "Fortaleza": "85",
                "Natal": "84",
                "João Pessoa": "83",
                "Recife": "81",
                "Maceió": "82",
                "Salvador": "71",
                "Aracaju": "79",
                "Teresina": "86",
                "São Luís": "98",
            }
            return f"{cidades[cidade]}{telefone}"
        return telefone
    df[telefone_col] = df.apply(lambda x: adicionar_ddd(x[telefone_col], x[cidade_col]), axis=1)
    df[telefone_col] = df[telefone_col].apply(lambda x: f"{x[:7]}-{x[7:]}")
    df[telefone_col] = df[telefone_col].apply(lambda x: f"({x[:2]}){x[2:]}")
    return df

In [7]:
# Função similar a função anetrior, porém que normatiza o telefone sem considerar a cidade.

def normalizar_telefone(df, telefone_col):
    df[telefone_col] = df[telefone_col].str.replace(' ', '', regex=False)
    df[telefone_col] = df[telefone_col].apply(lambda x: re.sub(r'\D', '', x))
    df[telefone_col] = df[telefone_col].apply(lambda x: x[1:] if len(x) == 12 or x.startswith('0') else x)
    def formatar_telefone(x):
        if len(x) == 8: 
            return f"{x[:4]}-{x[4:]}"
        elif len(x) == 9: 
            return f"{x[:5]}-{x[5:]}"
        elif len(x) == 10:
            return f"({x[:2]}){x[2:6]}-{x[6:]}"
        elif len(x) == 11:
            return f"({x[:2]}){x[2:7]}-{x[7:]}"
        else:
            return x 
    df[telefone_col] = df[telefone_col].apply(formatar_telefone)
    return df


In [8]:
# Função que altere a data do padrão americano AAAA-MM-DD para o padrão brasileiro DD/MM/AAAA

def normalizar_data(data):
    data = pd.to_datetime(data)
    return data.strftime("%d/%m/%Y")

In [9]:
# Função que normaliza o CNPJ de acordo com o padrão brasileiro XX.XXX.XXX/XXXX-XX

def normalizar_cnpj(cnpj):
    cnpj = cnpj.str.replace(r'\D', '', regex=True)
    cnpj = cnpj.apply(lambda x: f"{x[:2]}.{x[2:5]}.{x[5:8]}/{x[8:12]}-{x[12:]}")
    return cnpj

In [10]:
# Função de narmaliza a placa do veículo de acordo com o padrão brasileiro XXX-XXXX

def normalizar_placa(placa):
    placa = placa.str.replace(r'\W', '', regex=True)
    placa = placa.apply(lambda x: f"{x[:3]}-{x[3:]}")
    return placa

## 5. Aplicar as funções para transformar os dados em cada Dataframe

#### Utilizei as funções em cada Dataframe e de forma recorrente utilizei a função `head()` para verificar se os dados eram transformados da forma correta, novamente retirei a função, para melhorar a visibilidade do código.

In [11]:
clientes = normalizar_telefone_com_cidade(clientes, "telefone", "cidade")
clientes = normalizar_nome(clientes)
clientes["data_nascimento"] = clientes["data_nascimento"].apply(normalizar_data)


In [12]:
fornecedores["cnpj"] = normalizar_cnpj(fornecedores["cnpj"])
fornecedores["contato_responsavel"] = normalizar_nome(fornecedores["contato_responsavel"])
fornecedores = normalizar_telefone(fornecedores, "telefone")


In [13]:
funcionarios["nome_funcionario"] = normalizar_nome(funcionarios["nome_funcionario"])
funcionarios["data_contratacao"] = funcionarios["data_contratacao"].apply(normalizar_data)
funcionarios["data_nascimento"] = funcionarios["data_nascimento"].apply(normalizar_data)
funcionarios = normalizar_telefone(funcionarios, "telefone")


In [14]:
orcamentos["data_orcamento"] = orcamentos["data_orcamento"].apply(normalizar_data)

In [15]:
ordens_servico["data_abertura"] = ordens_servico["data_abertura"].apply(normalizar_data)
ordens_servico["data_fechamento"] = ordens_servico["data_fechamento"].apply(lambda x: x if pd.isnull(x) else normalizar_data(x))


In [16]:
veiculos["placa"] = normalizar_placa(veiculos["placa"])

## Salvar dataframes em arquivo Excel

#### Utilizei a função `ExcelWriter()` para salvar todos os Dataframes em um único arquivo em Excel nesta mesma pasta para iniciar o processo de DataViz com o Power BI. 

In [17]:
with pd.ExcelWriter("ultracar.xlsx") as writer:
    clientes.to_excel(writer, sheet_name="clientes", index=False)
    fornecedores.to_excel(writer, sheet_name="fornecedores", index=False)
    funcionarios.to_excel(writer, sheet_name="funcionarios", index=False)
    orcamentos.to_excel(writer, sheet_name="orcamentos", index=False)
    ordens_servico.to_excel(writer, sheet_name="ordens_servico", index=False)
    produtos.to_excel(writer, sheet_name="produtos", index=False)
    produtos_orcamento.to_excel(writer, sheet_name="produtos_orcamento", index=False)
    servicos.to_excel(writer, sheet_name="servicos", index=False)
    veiculos.to_excel(writer, sheet_name="veiculos", index=False)
    print("Arquivo salvo com sucesso!")


Arquivo salvo com sucesso!
